In [1]:
import json
import logging

from tqdm import tqdm

from Questionnaire.PsycoTest import UNIVERSAL_TRAITS
from AgentFactory import AgentFactory
from utils import clear_json

logging.basicConfig(level=logging.WARNING)

agent_factory = AgentFactory()

/home/youyuan/miniconda3/envs/transcreation/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# config_path: str = "configs/conf_LLaVA.json"
# config_path: str = "configs/conf_LLaMA3.json"
# config_path: str = "configs/conf_DeepSeek.json"
config_path: str = "configs/conf_GPT.json"
config = json.load(open(config_path, "r"))
config = config["model"]

agent = agent_factory.create_agent(config)

In [3]:
from Questionnaire.BigFive import BFI

qn = BFI()

ProgressBar = tqdm(total=len(qn.get_questions()), desc="Processing Questions")
for no, q in tqdm(enumerate(qn.get_questions())):
    ProgressBar.update(1)

    agent.reset_msg_history()
    agent.instruction = config["system_msg"]
    prompt = [
        {"type": "text", "text": q},
    ]
    response, text = agent.get_response(prompt)

    try:
        dict = json.loads(clear_json(text))
        qn.record_answer(no, dict)
    except Exception as e:
        print(f"Error: {e}")
        print(f"text: {text}")
        continue

final_output = qn.analyze()
qn.save_to_jsonl(f"output/BFI_{config['template_type']}.jsonl")

44it [01:06,  1.51s/it]00%|██████████| 44/44 [01:05<00:00,  1.81s/it]


In [7]:
from Questionnaire.HEXACO import HEXACO

hexaco = HEXACO()

ProgressBar = tqdm(total=len(hexaco.get_questions()), desc="Processing HEXACO Questions")
for no, q in tqdm(enumerate(hexaco.get_questions())):
    ProgressBar.update(1)

    agent.reset_msg_history()
    agent.instruction = config["system_msg"]
    prompt = [
        {"type": "text", "text": q},
    ]
    response, text = agent.get_response(prompt)

    try:
        dict = json.loads(clear_json(text))
        hexaco.record_answer(no, dict)
    except Exception as e:
        print(f"Error: {e}")
        print(f"text: {text}")
        continue

final_output_hex = hexaco.analyze()
hexaco.save_to_jsonl(f"output/HEXACO_{config['template_type']}.jsonl")

Processing Questions: 100%|██████████| 44/44 [02:42<00:00,  3.69s/it]
61it [01:43,  1.70s/it]


In [16]:
for k in UNIVERSAL_TRAITS:
    if k not in final_output_hex:
        print(f"{k}: None")
        continue
    print(f"{k}: {final_output_hex[k]}")
print("======================")
for k in UNIVERSAL_TRAITS:
    if k not in final_output:
        print(f"{k}: None")
        continue
    print(f"{k}: {final_output[k]}")

Extraversion: 4.125
Agreeableness: 4.1
Conscientiousness: 3.7
Emotional Stability: 3.5625
Openness: 4.428571428571429
Honesty Humility: 3.7
Extraversion: 4.25
Agreeableness: 3.6666666666666665
Conscientiousness: 3.7777777777777777
Emotional Stability: 3.625
Openness: 4.3
Honesty Humility: None


In [9]:
agent.llm.total_cost()

0.002265